<a href="https://colab.research.google.com/github/cagBRT/Diffusers/blob/main/stable_diffusion_seeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install the necessary libraries**

In [ ]:
!pip install huggingface huggingface_hub

In [ ]:
!pip install diffusers

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install transformers scipy ftfy

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

In [ ]:
from huggingface_hub import login
from diffusers import DiffusionPipeline
import torch

**Login to HuggingFace, get a write token**

In [ ]:
login()

**Setup the pipeline to use the stable-diffusion-vl-5 model**

In [ ]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)

In [ ]:
device = "cuda"
model_id = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True,
).to(device)

**A function to display the generated images**

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

**Set the constants**

In [ ]:
num_images = 4

width = 512
height = 512

# Latents Generation
In order to reuse the seeds we need to generate the latents ourselves. Otherwise, the pipeline will do it internally and we won't have a way to replicate them.

**Latents are the initial random Gaussian noise that gets transformed to actual images during the diffusion process.**

To generate them, we'll use a different random seed for each latent, and we'll save them so we can reuse them later.

In [ ]:
generator = torch.Generator(device=device)

latents = None
seeds = []
for _ in range(num_images):
    # Get a new random seed, store it and use it as the generator state
    seed = generator.seed()
    seeds.append(seed)
    generator = generator.manual_seed(seed)

    image_latents = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        device = device
    )
    latents = image_latents if latents is None else torch.cat((latents, image_latents))

# latents should have shape (4, 4, 64, 64) in this case
latents.shape

# Generate Images with our Latents

We are now ready to generate the images. **We'll send the pipeline the latents we want to use. If we don't, the pipeline will generate a new set for us.**

In [ ]:
prompt = "Labrador in the style of Vermeer"

with torch.autocast("cuda"):
    images = pipe(
        [prompt] * num_images,
        guidance_scale=7.5,
        latents = latents,
    )[0]

**Display the generated images**

In [ ]:
image_grid(images, 2, 2)

**Get the seed for the fourth image**

In [ ]:
seed = seeds[3]   # fourth one
seed

**We could just have reused `latents[3]` instead. But just taking note of the seeds will be enough te replicate the generation any time we like.**

In [ ]:
generator.manual_seed(seed)

latents = torch.randn(
    (1, pipe.unet.in_channels, height // 8, width // 8),
    generator = generator,
    device = device
)

If we repeat the generation (of a single image) with these latents and the same prompt, we should get the same image as before:

In [ ]:
with torch.autocast("cuda"):
    image = pipe(
        [prompt] * 1,
        guidance_scale=7.5,
        latents = latents,
    )[0]

image[0]

In [ ]:
prompt = "Terrier in the style of Vermeer"

with torch.autocast("cuda"):
    image = pipe(
        [prompt] * 1,
        guidance_scale=7.5,
        latents = latents,
    )[0]

image[0]

In [ ]:
prompt = "Labrador in the style of Van Gogh"

with torch.autocast("cuda"):
    image = pipe(
        [prompt] * 1,
        guidance_scale=7.5,
        latents = latents,
    )[0]

image[0]

In [ ]:
prompt = "Clown in the style of Vermeer"

with torch.autocast("cuda"):
    image = pipe(
        [prompt] * 1,
        guidance_scale=7.5,
        latents = latents,
    )[0]

image[0]